In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader

# Set the seed for reproducibility
torch.manual_seed(42)

# Define transforms for CIFAR-100
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Rescale for ResNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize to [-1, 1]
])

# Load CIFAR-100 dataset
dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
trainset, valset = random_split(dataset, [train_size, val_size])

# DataLoaders
batch_size = 32
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
import torchvision.models as models
import torch.nn as nn

# Load ResNet-18
model = models.resnet18(pretrained=True)  # Pretrained ResNet-18

# Adjust final layer for CIFAR-100
num_classes = 100
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
import torch.optim as optim
import torch.nn.functional as F

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    model.eval()
    file_name = "resnet18_" + str(epoch) + ".pth"
    torch.save(model.state_dict(), file_name)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(trainloader):.4f}")
    print("time ", datetime.now())

# Epoch 1/10, Loss: 2.4451
# time  2024-12-10 11:01:32.098552
# Epoch 2/10, Loss: 1.5218
# time  2024-12-10 11:15:45.045782
# Epoch 3/10, Loss: 1.1099
# time  2024-12-10 11:29:36.043563
# Epoch 4/10, Loss: 0.8012
# time  2024-12-10 11:43:18.962063
# Epoch 5/10, Loss: 0.5426
# time  2024-12-10 11:56:57.819060
# Epoch 6/10, Loss: 0.3654
# time  2024-12-10 12:10:23.815583
# Epoch 7/10, Loss: 0.2871
# time  2024-12-10 12:24:07.560979
# Epoch 8/10, Loss: 0.2257
# time  2024-12-10 12:37:37.487396
# Epoch 9/10, Loss: 0.1890
# time  2024-12-10 12:50:49.602340
# Epoch 10/10, Loss: 0.1942
# time  2024-12-10 13:03:58.092853